In [ ]:
from  lib.project_functions import * 

In [ ]:
train, val = generator(train=True, validation=True, generator_batch_size=128)


In [ ]:
train_sample = next(train)

print(train_sample[0].shape) 
print(train_sample[1].shape)  

In [ ]:
val_sample = next(val)

print(val_sample[0].shape) 
print(val_sample[1].shape)  

# Model

In [ ]:
convnext_tiny  = tf.keras.applications.convnext.ConvNeXtTiny(
                    model_name='convnext_tiny',
                    include_top=False,
                    include_preprocessing=True,
                    weights='imagenet',
                    input_tensor=None,
                    input_shape=(32, 32, 3) )

print( f"len -> {len(convnext_tiny.layers)}\ntype -> {type(convnext_tiny.layers)}")

In [ ]:
for layer in convnext_tiny.layers[:-20]:
    layer.trainable = False
for layer in convnext_tiny.layers:
    print(layer.name, layer.trainable)

In [ ]:
# convnext_tiny.summary()

In [ ]:
num_classes = int(pd.read_csv("data/Train.csv", usecols= ["ClassId"]).nunique())
num_classes

In [ ]:
inputs = tf.keras.Input(shape=(32, 32, 3))

convnext = convnext_tiny(inputs)
print("Shape of time_distributed_output:", convnext.shape)

flattened = Flatten()(convnext)

dance = layers.Dense(1024, activation='relu')(flattened)
dance = layers.Dense(512, activation='relu')(dance)
dance = layers.Dense(512, activation="relu")(dance)
print("Shape of dance:", dance.shape)

outputs = layers.Dense(num_classes, activation="softmax")(dance)

print("Shape of outputs:", outputs.shape)

convnext_model = tf.keras.Model(inputs=inputs, outputs=outputs)

convnext_model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', f1])

callback = EarlyStopping(
    patience=20,
    monitor="val_f1",
    restore_best_weights=True,
    min_delta=0.001
)

checkpoint = ModelCheckpoint(
    filepath = "modelcheckpoint/image_model.h5",  
    monitor='val_f1', 
    verbose=1,  
    save_best_only=True,  
    mode='min'  
)


hist = convnext_model.fit(train, epochs=1000, validation_data=val, verbose=1,
          callbacks=[callback, checkpoint])

In [ ]:
inputs = Input(shape=(32, 32, 3))

conv1 = Conv2D(32, (3, 3),  padding="same")(inputs)
conv1 = Activation('relu')(conv1)
conv1 = Conv2D(64, (3, 3),  padding="same")(conv1)
conv1 = Activation('relu')(conv1)
conv1 = Conv2D(128, (3, 3),  padding="same")(conv1)
conv1 = BatchNormalization()(conv1)
conv1 = Activation('relu')(conv1)
dropout1 = Dropout(0.3)(conv1)
pooling1 = MaxPooling2D()(dropout1)


conv2 = Conv2D(256, (3, 3),  padding="same")(pooling1)
conv2 = Activation('relu')(conv2)
conv2 = Conv2D(512, (3, 3),  padding="same")(conv2)
conv2 = Activation('relu')(conv2)
conv2 = Conv2D(512, (3, 3),  padding="same")(conv2)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)
dropout2 = Dropout(0.2)(conv2)
pooling2 = MaxPooling2D()(dropout2)

flattened = Flatten()(pooling2)
outputs = Dense(units=43, activation="softmax")(flattened)

model = Model(inputs=inputs, outputs=outputs)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=[f1])

hist = model.fit(train, epochs=1000, validation_data=val, verbose=1)



# VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam


# Загрузка предварительно обученной модели VGG16 без верхних слоев (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Добавление своего классификатора поверх базовой модели
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Создание окончательной модели
model = Model(inputs=base_model.input, outputs=predictions)

# Заморозка весов базовой модели
for layer in base_model.layers:
    layer.trainable = False

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=[f1])

# Вывод структуры модели
# model.summary()

model.fit(
        train,
        steps_per_epoch=len(train),
        epochs=1000,
        validation_data=val,
        validation_steps=len(val),
        callbacks=[callback, checkpoint])

In [ ]:
smmery = summarize_diagnostics(hist, save_path= "modelcheckpoint/image_model.h5.png")

In [ ]:
loaded_model = load_model("modelcheckpoint/image_model.h5")

In [ ]:
model_evaluate = loaded_model.evaluate(train)

# Test on my data 

In [ ]:
def my_foto(show = None):
    data_path = "data\MyFoto"
    my_foto_names = os.listdir(data_path)
    images = []
    
    for foto_name in my_foto_names:
        path = os.path.join(data_path, foto_name)
       
        image_cv = cv.imread(path)
        image_cv = cv.cvtColor(image_cv, cv.COLOR_BGR2RGB)
        # image_cv = image_preprocessing(image_cv)
        # resize image
        image_cv = cv.resize(image_cv, ( 32, 32), interpolation=cv.INTER_LINEAR)
        image_cv = image_cv / 255.0
        
        if show:
            image_show(image_cv)
            
        image_cv = np.expand_dims(image_cv, axis=0)  
        images.append(image_cv)
    
    return images

def predict(images, model ):
    predicted_classes = []
    for image in images:
        predicted_probabilities = model.predict(image)
        predicted_classes.append(np.argmax(predicted_probabilities, axis=1))
    return predicted_classes

images = my_foto( show = True)
# predicted = predict(images, loaded_model)
images[0].shape

In [ ]:
images[3].shape

In [ ]:
data_path = "data\Meta"
meta = os.listdir(data_path)

for klass, m, image in zip(predicted, meta, images):
    num_im_m = str(klass.item()) + ".png"
    path = os.path.join(data_path, num_im_m)
    image_2 = cv.imread(path)
    image_2 = cv.cvtColor(image_2, cv.COLOR_BGR2RGB)
    

    image_restored = np.squeeze(image, axis=0)  # Удаление добавленного измерения
    
    print(klass)
    image_show(image_2)
    image_show(image_restored)
    
    